In [42]:
import requests
import pandas as pd
import spacy
import ssl
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

In [43]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [44]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [45]:

def get_html_from_url(url):
  chrome_options = Options()
  chrome_options.add_argument("--headless")
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options) 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        flag = 0
  html = browser.page_source
  return html

In [46]:
# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")


In [47]:
prob_df = pd.read_csv(r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\articles\analyze_Articles\final_results\tough_urls\prob_urls.csv")

In [48]:
prob_df.columns

Index(['Unnamed: 0', 'links'], dtype='object')

In [49]:
prob_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [50]:
prob_urls = prob_df['links'].to_list()

In [51]:
prob_urls

['https://www.sciencedirect.com/science/article/abs/pii/S009265662200126X',
 'https://thehill.com/policy/energy-environment/3685583-nasa-suggests-new-space-cooling-technology-could-charge-electric-cars-in-5-minutes/',
 'https://newsroom.unsw.edu.au/news/science-tech/engineers-light-way-toward-bionics-future?utm_source=reddit&utm_medium=social',
 'https://techcrunch.com/2022/03/21/to-raise-a-fund-this-agtech-outfit-built-a-content-company-first-now-it-has-60-million-to-put-to-work/',
 'https://www.newsweek.com/baby-alpaca-learns-walk-again-prosthetic-leg-adorable-clip-1749410?',
 'https://www.sciencedirect.com/science/article/abs/pii/S0025326X22007627?via%3Dihub',
 'https://www.sciencedirect.com/science/article/abs/pii/S0925492722001172',
 'https://techcrunch.com/2022/03/01/zero-systems-gets-12m-series-a-to-bring-automation-to-professional-services/',
 'https://thehill.com/policy/energy-environment/3669504-supreme-court-to-hear-case-that-could-have-massive-impact-on-us-water-quality/',


In [53]:
#TODO get all missing abstracts
HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }




In [91]:
still_prob_links = []
cols_abstract_df = ['link', 'abstract']
abstract_df = pd.DataFrame([], columns=cols_abstract_df)

for i, url in enumerate(prob_urls):
    print(i)
    web = url.split('/')[2]
    result = requests.post(url, headers=HEADERS)
    text = 'UNKNOWN'

    soup = BeautifulSoup(result.text)

    if soup is None:
        still_prob_links.append(url)     
    
    if web == 'siliconangle.com':
        try:
            body = soup.find('div', {'class': 'single-post-content'}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)
    
    if web=='www.newsweek.com':
        try:
            body = soup.find('div', {'class':'article-body v_text'}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)
        
    if web=='www.thegamer.com':
        try:
            body = soup.find('section', {'id':'article-body'}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)
            
    if web=='apnews.com':
        try:
            body = soup.find('div', {"class":"Article"}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)

    if web=='www.uq.edu.au':
        try:
            body = soup.find('div', {'id':"content"}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)
    
    if web=='www.wsj.com':
        try:
            text = soup.find('section', {'class':'css-az2xkl-Container-Container e1d75se20'}).text
        except:
            still_prob_links.append(url)
    
    if web=='www.tandfonline.com':
        try:
            text = soup.find("div", {"class":"abstractSection abstractInFull"}).text
        except:
            still_prob_links.append(url)
    
    if web=='www.abc.net.au':
        try:
            body = soup.find("div", {"class":"_3P3cP _3sFAh"}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)
            
    if web=='www.tomshardware.com':
        try:
            body = soup.find("div",{"id":"article-body"})
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)
        
    if web == 'thehill.com':
        try:
            body = soup.find("div", {"class":"article__text | body-copy | flow"}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)

    if web=='newsroom.unsw.edu.au':
        try:
            body = soup.find("div", {"class":"field-article-body"})
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)


    if web=='techcrunch.com':
        try:
            body = soup.find("div", {"class":"article-content"})
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)
    
    if web=='eandt.theiet.org':
        try:
            body = soup.find("div", {"class":"body-content"})
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)

    if web=='cosmosmagazine.com':
        try:
            body = soup.find("div", {"class":"copy"})
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)
    
    if web=='cleantechnica.com':
        try:
            body = soup.find("div", {"class":"zox-post-body left zoxrel zox100"})
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)

    if web=='www.eurekalert.org':
        try:
            body = soup.find("div", {"class":"entry"})
            text = "".join([p.find_next(text=True).strip() for p in body])
        except:
            still_prob_links.append(url)

    if web=='arstechnica.com':
        try:
            all_pees = soup.find_all('p')
            text ="".join([p.find_next(text=True).strip() for p in all_pees])
        except:
            still_prob_links.append(url)

    temp_df = pd.DataFrame([[url, text]], columns=cols_abstract_df)
    abstract_df = pd.concat([abstract_df, temp_df], axis=0)        


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [92]:
len(still_prob_links)

5

In [93]:
still_prob_links

['https://apnews.com/article/elon-musk-technology-business-artificial-intelligence-tesla-inc-217a2a3320bb0f2e78224994f15ffb11?utm_source=homepage&utm_medium=TopNews&utm_campaign=position_09',
 'https://apnews.com/article/technology-european-union-commission-f981003838097a36ff90fc66afeb2d63',
 'https://www.wsj.com/articles/chinas-factories-accelerate-robotics-push-as-workforce-shrinks-11663493405',
 'https://www.abc.net.au/news/science/2022-10-26/extreme-miyake-radiation-events-tree-rings-solar-storms/101563738',
 'https://www.wsj.com/articles/solar-rollout-rouses-resistance-in-europes-countryside-11665234001']

In [94]:
abstract_df

,link,abstract
0,https://www.sciencedirect.com/science/article/...,UNKNOWN
0,https://thehill.com/policy/energy-environment/...,NASA has suggested an experimental cooling sys...
0,https://newsroom.unsw.edu.au/news/science-tech...,Biomedical and electrical engineers at UNSW Sy...
0,https://techcrunch.com/2022/03/21/to-raise-a-f...,Rob Leclerc has the kind of pedigree that inve...
0,https://www.newsweek.com/baby-alpaca-learns-wa...,An alpaca has been given a second chance at li...
...,...,...
0,https://www.science.org/doi/10.1126/sciadv.add...,UNKNOWN
0,https://techcrunch.com/2022/03/16/legal-tech-s...,It’s long known that the legal profession has ...
0,https://www.science.org/doi/10.1126/sciadv.abq...,UNKNOWN
0,https://arstechnica.com/tech-policy/2022/10/co...,Front page layoutSite themeProPublica is a Pul...


In [95]:
#add items to abstracts full
path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\articles\analyze_Articles\final_results\abstracts_with_prob_urls.csv"
df1 = pd.read_csv(path)



In [96]:
len(df1)

642

In [97]:
df1 = pd.concat([df1, abstract_df], axis=0)

In [98]:
df1

,Unnamed: 0,link,abstract
0,0.0,http://jhr.uwpress.org/content/early/2022/10/0...,We study how ambient lead exposure impacts lea...
1,1.0,http://www.exeter.ac.uk/news/research/title_93...,Positive childhood experiences of blue spaces ...
2,2.0,https://9to5mac.com/2022/10/25/apple-watch-blo...,A new validation study published this month pu...
3,3.0,https://9to5mac.com/2022/10/27/apple-cant-make...,Apple’s fiscal Q4 earnings are out and while t...
4,4.0,https://aacrjournals.org/clincancerres/article...,Combustible tobacco use has reached historic l...
...,...,...,...
0,NaN,https://www.science.org/doi/10.1126/sciadv.add...,UNKNOWN
0,NaN,https://techcrunch.com/2022/03/16/legal-tech-s...,It’s long known that the legal profession has ...
0,NaN,https://www.science.org/doi/10.1126/sciadv.abq...,UNKNOWN
0,NaN,https://arstechnica.com/tech-policy/2022/10/co...,Front page layoutSite themeProPublica is a Pul...


In [104]:
df1 = df1[df1['abstract']!='UNKNOWN']

In [105]:
new_path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\articles\analyze_Articles\final_results\abstracts_full_final.csv"
df1.to_csv(new_path)